## Exercício Prático

### Faça o download do data set ml-100k do movielens e importe dentro da pasta resources. 

### Crie uma pasta com o seu nome dentro de /user no HDFS

In [14]:
!wget http://files.grouplens.org/datasets/movielens/ml-latest-small.zip -q --show-progress
!unzip ml-latest-small.zip
!rm ml-latest-small.zip

ml-latest-small.zip 100%[===================>] 955.28K   918KB/s    in 1.0s    
Archive:  ml-latest-small.zip
   creating: ml-latest-small/
  inflating: ml-latest-small/links.csv  
  inflating: ml-latest-small/tags.csv  
  inflating: ml-latest-small/ratings.csv  
  inflating: ml-latest-small/README.txt  
  inflating: ml-latest-small/movies.csv  


In [17]:
%env DATASET_PATH ml-latest-small

env: DATASET_PATH=ml-latest-small


In [18]:
!ls ${DATASET_PATH} -lha

total 3.2M
drwxr-xr-x 2 jovyan root   4.0K Sep 26  2018 .
drwxr-xr-x 1 jovyan jovyan 4.0K Sep 24 23:01 ..
-rw-r--r-- 1 jovyan root   194K Sep 26  2018 links.csv
-rw-r--r-- 1 jovyan root   483K Sep 26  2018 movies.csv
-rw-r--r-- 1 jovyan root   2.4M Sep 26  2018 ratings.csv
-rw-r--r-- 1 jovyan root   8.2K Sep 26  2018 README.txt
-rw-r--r-- 1 jovyan root   116K Sep 26  2018 tags.csv


In [19]:
!hdfs dfs -ls /

Found 3 items
drwxr-xr-x   - jovyan supergroup          0 2020-09-24 22:43 /hbase
drwxrwxr-x   - jovyan supergroup          0 2020-09-24 22:42 /tmp
drwxr-xr-x   - jovyan supergroup          0 2020-09-24 22:48 /user


In [5]:
!hdfs dfs -mkdir /user/murilo

In [7]:
!hdfs dfs -ls /user

Found 3 items
drwxr-xr-x   - jovyan supergroup          0 2020-09-24 22:42 /user/hive
drwxr-xr-x   - jovyan supergroup          0 2020-09-24 22:42 /user/jovyan
drwxr-xr-x   - jovyan supergroup          0 2020-09-24 22:48 /user/murilo


### Copie para o HDFS, dentro da pasta com seu nome, o arquivo que possui os ratings dos usuários para os filmes 

In [22]:
!hadoop fs -copyFromLocal ${DATASET_PATH}/ratings.csv /user/murilo

In [23]:
!hdfs dfs -ls /user/murilo

Found 1 items
-rw-r--r--   1 jovyan supergroup    2483723 2020-09-24 23:04 /user/murilo/ratings.csv


## Utilize MapReduce para responder às perguntas abaixo:

### Qual a quantidade de ratings para cada nível? 

Execute o seu código no sistema de arquivos local e no HDFS -- utilize Hadoop Streaming 

In [17]:
!find $HOME/ -name hadoop-streaming*.jar

/home/jovyan/resources/local/hadoop-2.9.2/share/hadoop/tools/sources/hadoop-streaming-2.9.2-sources.jar
/home/jovyan/resources/local/hadoop-2.9.2/share/hadoop/tools/sources/hadoop-streaming-2.9.2-test-sources.jar
/home/jovyan/resources/local/hadoop-2.9.2/share/hadoop/tools/lib/hadoop-streaming-2.9.2.jar


In [24]:
# !cat ./resources/datasets/animals.txt | python resources/mappers/mapper1.py | sort -k1,1 | python resources/reducers/reducer1.py
!hdfs dfs -rm /user/murilo/output/*
!hdfs dfs -rmdir /user/murilo/output/

!hadoop jar $HADOOP_HOME/share/hadoop/tools/lib/hadoop-streaming-2.9.2.jar \
-input   /user/murilo/ratings.csv \
-output  /user/murilo/output \
-mapper  "python $(pwd)/resources/mappers/mapper1.py" \
-reducer "python $(pwd)/resources/reducers/reducer1.py"


Deleted /user/murilo/output/_SUCCESS
Deleted /user/murilo/output/part-00000
packageJobJar: [/tmp/hadoop-unjar6143247490730750364/] [] /tmp/streamjob2733331306892472727.jar tmpDir=null
20/09/25 00:15:24 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
20/09/25 00:15:25 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
20/09/25 00:15:25 INFO mapred.FileInputFormat: Total input files to process : 1
20/09/25 00:15:25 INFO mapreduce.JobSubmitter: number of splits:2
20/09/25 00:15:25 INFO Configuration.deprecation: yarn.resourcemanager.system-metrics-publisher.enabled is deprecated. Instead, use yarn.system-metrics-publisher.enabled
20/09/25 00:15:25 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1600987335947_0009
20/09/25 00:15:26 INFO impl.YarnClientImpl: Submitted application application_1600987335947_0009
20/09/25 00:15:26 INFO mapreduce.Job: The url to track the job: http://jupyter-thedatasociety-2dlab-2dhadoop-2di77mtacu:8088/proxy/app

rm: `/user/murilo/output/_temporary': Is a directory


In [3]:
!hdfs dfs -ls /user/murilo/output

Found 2 items
-rw-r--r--   1 jovyan supergroup          0 2020-09-24 23:48 /user/murilo/output/_SUCCESS
-rw-r--r--   1 jovyan supergroup    2584560 2020-09-24 23:48 /user/murilo/output/part-00000


In [25]:
!hdfs dfs -tail /user/murilo/output/part-00000

nivel 0.5	1370
nivel 1.0	2811
nivel 1.5	1791
nivel 2.0	7551
nivel 2.5	5550
nivel 3.0	20047
nivel 3.5	13136
nivel 4.0	26818
nivel 4.5	8551
nivel 5.0	13211
nivel rating	1


### Qual a popularidade de cada filme?

Execute o seu código no sistema de arquivos local e no HDFS -- utilize Hadoop Streaming 

In [28]:
# !cat ./resources/datasets/animals.txt | python resources/mappers/mapper1.py | sort -k1,1 | python resources/reducers/reducer1.py
!hdfs dfs -rm /user/murilo/outputEx2/*
!hdfs dfs -rmdir /user/murilo/outputEx2/

!hadoop jar $HADOOP_HOME/share/hadoop/tools/lib/hadoop-streaming-2.9.2.jar \
-input   /user/murilo/ratings.csv \
-output  /user/murilo/outputEx2 \
-mapper  "python $(pwd)/resources/mappers/mapper2.py" \
-reducer "python $(pwd)/resources/reducers/reducer1.py"


Deleted /user/murilo/outputEx2/_SUCCESS
Deleted /user/murilo/outputEx2/part-00000
packageJobJar: [/tmp/hadoop-unjar2234315130812889202/] [] /tmp/streamjob1998187796857259312.jar tmpDir=null
20/09/25 00:26:58 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
20/09/25 00:26:59 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
20/09/25 00:26:59 INFO mapred.FileInputFormat: Total input files to process : 1
20/09/25 00:27:00 INFO mapreduce.JobSubmitter: number of splits:2
20/09/25 00:27:00 INFO Configuration.deprecation: yarn.resourcemanager.system-metrics-publisher.enabled is deprecated. Instead, use yarn.system-metrics-publisher.enabled
20/09/25 00:27:00 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1600987335947_0011
20/09/25 00:27:00 INFO impl.YarnClientImpl: Submitted application application_1600987335947_0011
20/09/25 00:27:01 INFO mapreduce.Job: The url to track the job: http://jupyter-thedatasociety-2dlab-2dhadoop-2di77mtacu:8088/pro

In [30]:
!hdfs dfs -tail /user/murilo/outputEx2/part-00000

	31
nivel 97950	1
nivel 97988	1
nivel 98061	1
nivel 98083	2
nivel 981	2
nivel 98122	2
nivel 98124	7
nivel 98154	4
nivel 98160	1
nivel 98175	1
nivel 982	1
nivel 98203	4
nivel 98230	1
nivel 98239	2
nivel 98243	12
nivel 98279	2
nivel 98296	1
nivel 98361	1
nivel 984	2
nivel 98491	8
nivel 98499	2
nivel 98503	1
nivel 98585	5
nivel 986	13
nivel 98604	1
nivel 98607	1
nivel 98623	2
nivel 98633	1
nivel 98697	1
nivel 987	1
nivel 98799	1
nivel 988	2
nivel 98809	40
nivel 98836	1
nivel 98908	1
nivel 98961	14
nivel 99	2
nivel 990	5
nivel 99005	1
nivel 99007	14
nivel 99030	1
nivel 99087	1
nivel 991	11
nivel 99106	3
nivel 99112	12
nivel 99114	71
nivel 99117	8
nivel 99122	1
nivel 99130	1
nivel 99145	3
nivel 99149	7
nivel 99191	1
nivel 993	1
nivel 994	11
nivel 99415	1
nivel 99437	2
nivel 99532	2
nivel 99574	1
nivel 996	13
nivel 99636	1
nivel 99638	1
nivel 99721	1
nivel 99728	3
nivel 99750	1
nivel 99764	2
nivel 998	3
nivel 99813	8
nivel 99846	1
nivel 99853	1
nivel 999	12
nivel 99910	2
nivel 99917	3
nivel 